通过这个系列教程一步一步学习如何使用更小更快的 Rollup 取代 Webpack 和 Browserify 打包 JavaScript 文件。

这周，我们要使用Rollup构建我们的第一个项目，Rollup 是一个打包 JavaScript(和样式，不过下周才会做)的构建工具。

通过这个教程，我们的Rollup将能够：

* 合并scripts代码，
* 删除多余代码，
* 编译成对旧浏览器友好的代码，
* 支持在浏览器中使用Node模块，
* 能使用环境变量，
* 尽可能的压缩，减少文件大小。

## 准备工作

至少懂一点JavaScript的话将会更好理解。
对 [ES2015 modules](https://github.com/getify/You-Dont-Know-JS/blob/master/es6%20%26%20beyond/ch3.md#modules) 有基本了解，不过不了解也无妨。
在你的设备上要有npm。(还没有？在这下载Node.js)

## Rollup是什么？

用他们自己的话说：

> Rollup是下一代JavaScript模块打包工具。开发者可以在你的应用或库中使用ES2015模块，然后高效地将它们打包成一个单一文件用于浏览器和Node.js使用。

和 [Browserify](http://browserify.org/) 和 [Webpack](https://webpack.github.io/) 很像。

你也可以认为Rollup是一个构建工具，可以和像 [Grunt](http://gruntjs.com/) 和 [Gulp](https://github.com/gulpjs/gulp) 等一起配置使用。但是，需要注意的一点是当你使用 [Grunt](http://gruntjs.com/) 和 [Gulp](https://github.com/gulpjs/gulp) 来处理像打包 JavaScript 这样的任务时，这些工具的底层还是使用了像 Rollup，[Browserify](http://browserify.org/) 或 [Webpack](https://webpack.github.io/) 这些东西。

## 为什么应该关注Rollup？

Rollup 最令人激动的地方，就是能让打包文件体积很小。这么说很难理解，更详细的解释：相比其他 JavaScript 打包工具，Rollup 总能打出更小，更快的包。

因为 Rollup 基于 ES2015 模块，比 [Webpack](https://webpack.github.io/) 和 [Browserify](http://browserify.org/) 使用的 CommonJS 模块机制更高效。这也让 Rollup 从模块中删除无用的代码，即 `tree-shaking` 变得更容易。

当我们引入拥有大量函数和方法的三方工具或者框架时 `tree-shaking` 会变得很重要。想想 lodash 或者 jQuery，如果我们只使用一个或者两个方法，就会因为加载其余内容而产生大量无用的开销。

[Browserify](http://browserify.org/) 和 [Webpack](https://webpack.github.io/) 就会包含大量无用的代码。但是 Rollup 不会 - 它只会包括我们真正用到的东西。

更新 (2016-08-22): 澄清一下，Rollup 只能对ES模块上进行 `tree-shaking`。CommonJS 模块 - 像 lodash 和 jQuery 那样写的模块不能进行 `tree-shaking`。然而，`tree-shaking` 不是Rollup 在速度/性能上唯一的优势。可以看 [Rich Harris 的解释](https://www.reddit.com/r/javascript/comments/4yprc5/how_to_bundle_javascript_with_rollup_stepbystep/d6qzgzm) 和 [Nolan Lawson](https://www.reddit.com/r/javascript/comments/4yprc5/how_to_bundle_javascript_with_rollup_stepbystep/d6qzmgh?context=3) 的补充了解更多。

还有一个大新闻。

注意: 由于 Rollup 很高效，[Webpack](https://webpack.github.io/) 也将支持 `tree-shaking`。

## 如何使用Rollup处理并打包JavaScript文件

为了展示Rollup如何使用，让我们通过构建一个简单的项目来走一遍使用 Rollup 打包 JavaScript 的过程。

### 创建一个包含将被编译的JavaScript和CSS的项目

为了开始工作，我们需要一些用来处理的代码。这个教程里，我们将用一个小应用，可从GitHub获取。

目录结构如下：

```
learn-rollup/
├── src/
│   ├── scripts/
│   │   ├── modules/
│   │   │   ├── mod1.js
│   │   │   └── mod2.js
│   │   └── main.js
│   └── styles/
│       └── main.css
└── package.json
```

你可以在终端执行下面的命令下载这个应用，我们将在这个教程中使用它。

```bash
# Move to the folder where you keep your dev projects.
cd /path/to/your/projects

# Clone the starter branch of the app from GitHub.
git clone -b step-0 --single-branch https://github.com/jlengstorf/learn-rollup.git

# The files are downloaded to /path/to/your/projects/learn-rollup/
```

### 安装Rollup并且创建配置文件

第一步，执行下面的命令安装 Rollup：

```bash
npm install --save-dev rollup
```

然后，在 learn-rollup 文件夹下新建`rollup.config.js`。在文件中添加如下内容。

```js
export default {
  entry: 'src/scripts/main.js',
  dest: 'build/js/main.min.js',
  format: 'iife',
  sourceMap: 'inline',
};
```

说说每个配置项实际上做了什么：

* `entry` — 希望Rollup处理的文件路径。大多数应用中，它将是入口文件，初始化所有东西并启动应用。
* `dest` — 编译完的文件需要被存放的路径。
* `format` — Rollup支持多种输出格式。因为我们是要在浏览器中使用，需要使用立即执行函数表达式(IIFE)[注1]
* `sourceMap` — 调试时sourcemap是非常有用的。这个配置项会在生成文件中添加一个sourcemap，让开发更方便。

> NOTE: 对于其他的 `format` 选项以及你为什么需要他们，看 [Rollup’s wiki](https://github.com/rollup/rollup/wiki/JavaScript-API#format)。

### 测试 Rollup 配置

当创建好配置文件后，在终端执行下面的命令测试每项配置是否工作：

```bash
./node_modules/.bin/rollup -c
```

在你的项目下会出现一个`build`目录，包含`js`子目录，子目录中包含生成的`main.min.js`文件。

在浏览器中打开`build/index.html`可以看到打包文件正确生成了。



完成第一步后我们的示例项目的状态。

> 注意：现在，只有现代浏览器下不会报错。为了能够在不支持ES2015/ES6的老浏览器中运行，我们需要添加一些插件。

### 看看打包出来的文件

事实上 Rollup 强大是因为它使用了“tree-shaking”，可以在你引入的模块中删除没有用的代码。举个例子，在 src/scripts/modules/mod1.js 中的 sayGoodbyeTo() 函数在我们的应用中并没有使用 - 而且因为它从不会被使用，Rollup 不会将它打包到 bundle 中：

In [ ]:
(function () {
'use strict';

/**
 * Says hello.
 * @param  {String} name a name
 * @return {String}      a greeting for `name`
 */
function sayHelloTo( name ) {
  const toSay = `Hello, ${name}!`;

  return toSay;
}

/**
 * Adds all the values in an array.
 * @param  {Array} arr an array of numbers
 * @return {Number}    the sum of all the array values
 */
const addArray = arr => {
  const result = arr.reduce((a, b) => a + b, 0);

  return result;
};

// Import a couple modules for testing.
// Run some functions from our imported modules.
const result1 = sayHelloTo('Jason');
const result2 = addArray([1, 2, 3, 4]);

// Print the results on the page.
const printTarget = document.getElementsByClassName('debug__output')[0];

printTarget.innerText = `sayHelloTo('Jason') => ${result1}\n\n`
printTarget.innerText += `addArray([1, 2, 3, 4]) => ${result2}`;

}());
//# sourceMappingURL=data:application/json;charset=utf-8;base64,...

其他的构建工具则不是这样的，如果我们引入了一个像lodash这样一个很大的库而只是使用其中一两个函数时，我们的包文件会变得非常大。

比如使用 Webpack 的话，sayGoodbyeTo() 也会打包进去，产生的打包文件比 Rollup 生成的大了两倍多。

## 配置 babel 支持 JavaScript 新特性

现在我们已经得到能在现代浏览器中运行的包文件了，但是在一些旧版本浏览器中就会崩溃 - 这并不理想。

幸运的是，[Babel](http://babeljs.io/) 已发布了。这个项目编译 JavaScript 新特性(ES6/ES2015等等)到 ES5, 差不多在今天的任何浏览器上都能运行。

如果你还没用过 [Babel](http://babeljs.io/)，那么你的开发生涯要永远地改变了。使用 JavaScript 的新方法让语言更简单，更简洁而且整体上更友好。

那么让我们为 Rollup 加上这个过程，我们就不用担心上面的问题了。

### 安装必要模块

首先，我们需要安装 [Babel Rollup plugin](https://github.com/rollup/rollup-plugin-babel) 和适当的 Babel preset。

```bash
# Install Rollup’s Babel plugin.
npm install --save-dev rollup-plugin-babel

# Install the Babel preset for transpiling ES2015 using Rollup.
npm install --save-dev babel-preset-es2015-rollup
```

提示: Babel preset 是告诉 Babel 我们实际需要哪些 babel 插件的集合。

### 创建 `.babelrc`

然后，在项目根目录(learn-rollup/)下创建一个 .babelrc 文件。在文件中添加下面的 JSON：

```json
{
  "presets": ["es2015-rollup"],
}
```

它会告诉 Babel 在转换时哪些 preset 将会用到。

### 更新rollup.config.js

为了让它能真正工作，我们需要更新 `rollup.config.js`。

在文件中，importBabel 插件，将它添加到新配置属性plugins中，这个属性接收一个插件组成的数组。

```js
// Rollup plugins
import babel from 'rollup-plugin-babel';

export default {
  entry: 'src/scripts/main.js',
  dest: 'build/js/main.min.js',
  format: 'iife',
  sourceMap: 'inline',
  plugins: [
    babel({
      exclude: 'node_modules/**',
    }),
  ],
};
```

为避免编译三方脚本，通过设置exclude属性忽略node_modules目录。

### 检查输出文件

全部都安装并配置好后，重新打包一下：

```bash
./node_modules/.bin/rollup -c
```

再看一下输出结果，大部分是一样的。但是有一些地方不一样：比如，addArray()这个函数：

```js
var addArray = function addArray(arr) {
  var result = arr.reduce(function (a, b) {
    return a + b;
  }, 0);

  return result;
};
```

Babel 是如何将箭头函数 `(arr.reduce((a, b) => a + b, 0))` 转换成一个普通函数的呢？

这就是编译的意义：结果是相同的，但是现在的代码可以向后支持到IE9.

> 注意: Babel也提供了babel-polyfill,使得像Array.prototype.reduce()这些方法在IE8甚至更早的浏览器也能使用。

## 添加ESLint检查常规JavaScript错误

在你的项目中使用 linter 是个好主意，因为它强制统一了代码风格并且能帮你发现很难找到的bug，比如花括号或者圆括号。

在这个项目中，我们将使用 [ESLint](http://eslint.org/)。

### 安装模块

为使用 [ESLint](http://eslint.org/)，我们需要安装 [ESLint Rollup plugin](https://github.com/TrySound/rollup-plugin-eslint)：

```bash
npm install --save-dev rollup-plugin-eslint
```

### 生成一个.eslintrc.json

为确保我们只得到我们想检测的错误，首先要配置 [ESLint](http://eslint.org/)。很幸运，我们可以通过执行下面的命令自动生成大多数配置：

```bash
./node_modules/.bin/eslint --init
? How would you like to configure ESLint? Answer questions about your style
? Are you using ECMAScript 6 features? Yes
? Are you using ES6 modules? Yes
? Where will your code run? Browser
? Do you use CommonJS? No
? Do you use JSX? No
? What style of indentation do you use? Spaces
? What quotes do you use for strings? Single
? What line endings do you use? Unix
? Do you require semicolons? Yes
? What format do you want your config file to be in? JSON
Successfully created .eslintrc.json file in /Users/jlengstorf/dev/code.lengstorf.com/projects/learn-rollup
```

如果你按上面展示的那样回答问题，你将在生成的 `.eslintrc.json` 中得到下面的内容：

```json
{
  "env": {
    "browser": true,
    "es6": true
  },
  "extends": "eslint:recommended",
  "parserOptions": {
    "sourceType": "module"
  },
  "rules": {
    "indent": [
      "error",
      4
    ],
    "linebreak-style": [
      "error",
      "unix"
    ],
    "quotes": [
      "error",
      "single"
    ],
    "semi": [
      "error",
      "always"
    ]
  }
}
```

### 修改.eslintrc.json

然而我们需要改动两个地方来避免项目报错。

使用2空格代替4空格。

后面会使用到ENV这个全局变量，因此要把它加入白名单中。

在 `.eslintrc.json` 进行如下修改 — 添加 `globals` 属性并修改 `indent` 属性：

```json
{
  "env": {
    "browser": true,
    "es6": true
  },
  "globals": {
    "ENV": true
  },
  "extends": "eslint:recommended",
  "parserOptions": {
    "sourceType": "module"
  },
  "rules": {
    "indent": [
      "error",
      2
    ],
    "linebreak-style": [
      "error",
      "unix"
    ],
    "quotes": [
      "error",
      "single"
    ],
    "semi": [
      "error",
      "always"
    ]
  }
}
```

### 更新rollup.config.js

然后，引入ESLint插件并添加到Rollup配置中：

```js
// Rollup plugins
import babel from 'rollup-plugin-babel';
import eslint from 'rollup-plugin-eslint';

export default {
  entry: 'src/scripts/main.js',
  dest: 'build/js/main.min.js',
  format: 'iife',
  sourceMap: 'inline',
  plugins: [
    babel({
      exclude: 'node_modules/**',
    }),
    eslint({
      exclude: [
        'src/styles/**',
      ]
    }),
  ],
};
```

### 检查控制台的输出

第一次，当执行 `./node_modules/.bin/rollup -c` 时，似乎什么都没发生。因为这表示应用的代码通过了linter，没有问题。

但是如果我们制造一个错误 - 比如删除一个分号 - 我们会看到 [ESLint](http://eslint.org/) 是如何提示的：

```bash
./node_modules/.bin/rollup -c

/Users/jlengstorf/dev/code.lengstorf.com/projects/learn-rollup/src/scripts/main.js
  12:64  error  Missing semicolon  semi

✖ 1 problem (1 error, 0 warnings)
```

一些包含潜在风险和解释神秘 bug 的东西立刻出现了，包括出现问题的文件，行和列。

但是它不能排除我们调试时的所有问题，很多由于拼写错误和疏漏产生的bug还是要自己花时间去解决。

## 添加插件处理非ES模块

如果你的依赖中有任何使用 Node 风格的模块这个插件就很重要。如果没有它，你会得到关于 require 的错误。

添加一个 Node 模块作为依赖。

在这个小项目中不引用三方模块很正常，但实际项目中不会如此。所以为了让我们的 Rollup 配置变得真正可用，需要保证在我们的代码中也能引用是三方模块。

举个简单的例子，我们将使用 debug 包添加一个简单的日志打印器到项目中。先安装它：

```bash
npm install --save debug
```

> 注意：因为它是会在主程序中引用的，应该使用 `--save` 参数，可以避免在生产环境下出现错误，因为 `devDependencies` 在生产环境下不会被安装。

然后在 `src/scripts/main.js` 中添加一个简单的日志：

```js
// Import a couple modules for testing.
import { sayHelloTo } from './modules/mod1';
import addArray from './modules/mod2';

// Import a logger for easier debugging.
import debug from 'debug';
const log = debug('app:log');

// Enable the logger.
debug.enable('*');
log('Logging is enabled!');

// Run some functions from our imported modules.
const result1 = sayHelloTo('Jason');
const result2 = addArray([1, 2, 3, 4]);

// Print the results on the page.
const printTarget = document.getElementsByClassName('debug__output')[0];

printTarget.innerText = `sayHelloTo('Jason') => ${result1}\n\n`;
printTarget.innerText += `addArray([1, 2, 3, 4]) => ${result2}`;
```

到此一切都很好，但是当运行 rollup 时会得到一个警告：

```bash
./node_modules/.bin/rollup -c
Treating 'debug' as external dependency
No name was provided for external module 'debug' in options.globals – guessing 'debug'
```

而且如果在查看 index.html，会发现一个 ReferenceError 抛出了：



默认情况下，三方的 Node 模块无法在Rollup中正确加载。

哦，真糟糕。完全无法运行。

因为 Node 模块使用 CommonJS，无法与 Rollup 直接兼容。为解决这个问题，需要添加一组处理 Node 模块和 CommonJS 模块的插件。

### 安装模块

围绕这个问题，我们将在 Rollup 中新增两个插件：

* rollup-plugin-node-resolve，运行加载node_modules中的三方模块。
* rollup-plugin-commonjs，将CommonJS模块转换成ES6，防止他们在Rollup中失效。

通过下面的命令安装两个插件：

```bash
npm install --save-dev rollup-plugin-node-resolve rollup-plugin-commonjs
```

### 更新 rollup.config.js

然后，引入插件并添加进 Rollup 配置：

```js
// Rollup plugins
import babel from 'rollup-plugin-babel';
import eslint from 'rollup-plugin-eslint';
import resolve from 'rollup-plugin-node-resolve';
import commonjs from 'rollup-plugin-commonjs';

export default {
  entry: 'src/scripts/main.js',
  dest: 'build/js/main.min.js',
  format: 'iife',
  sourceMap: 'inline',
  plugins: [
    resolve({
      jsnext: true,
      main: true,
      browser: true,
    }),
    commonjs(),
    eslint({
      exclude: [
        'src/styles/**',
      ]
    }),
    babel({
      exclude: 'node_modules/**',
    }),
  ],
};
```

> 注意: jsnext属性是为了帮助Node模块迁移到ES2015的一部分。main和browser 属性帮助插件决定哪个文件应该被bundle文件使用。

### 检查控制台输出

执行 `./node_modules/.bin/rollup -c` 重新打包，然后再检查浏览器输出：



成功了！日志现在打印出来了。

添加插件替换环境变量
环境变量使开发流程更强大，让我们有能力做一些事情，比如打开或关闭日志，注入仅在开发环境使用的脚本等等。

那么让Rollup支持这些功能吧。

## 在main.js中添加ENV变量

让我们通过一个环境变量控制日志脚本，让日志脚本只能在非生产环境下使用。在src/scripts/main.js中修改log()的初始化方式。

```js
// Import a logger for easier debugging.
import debug from 'debug';
const log = debug('app:log');

// The logger should only be disabled if we’re not in production.
if (ENV !== 'production') {

  // Enable the logger.
  debug.enable('*');
  log('Logging is enabled!');
} else {
  debug.disable();
}
```

然而，重新打包(`./node_modules/.bin/rollup -c`)后检查浏览器，会看到一个 ENV 的ReferenceError。

不必惊讶，因为我们没有在任何地方定义它。如果我们尝试`ENV=production ./node_modules/.bin/rollup -c`，还是不会成功。因为那样设置的环境变量只是在 Rollup 中可用，不是在 Rollup 打包的 bundle 中可用。

我们需要使用一个插件将环境变量传入bundle。

### 安装模块

安装 `rollup-plugin-replace` 插件，它本质上只是做了查找-替换的工作。它能做很多事情，但现在我们只需要让它简单地找到出现的环境变量并将其替换成实际的值。（比如，所有在 bundle 出现的 ENV 变量都会被替换成 "production" ）。

```bash
npm install --save-dev rollup-plugin-replace
```

### 更新rollup.config.js

在 rollup.config.js 中引入插件并且添加到插件列表中。

配置非常简单：只需添加一个键值对的列表，key 是将被替换的字符串，value 是应该被替换成的值。

```js
// Rollup plugins
import babel from 'rollup-plugin-babel';
import eslint from 'rollup-plugin-eslint';
import resolve from 'rollup-plugin-node-resolve';
import commonjs from 'rollup-plugin-commonjs';
import replace from 'rollup-plugin-replace';

export default {
  entry: 'src/scripts/main.js',
  dest: 'build/js/main.min.js',
  format: 'iife',
  sourceMap: 'inline',
  plugins: [
    resolve({
      jsnext: true,
      main: true,
      browser: true,
    }),
    commonjs(),
    eslint({
      exclude: [
        'src/styles/**',
      ]
    }),
    babel({
      exclude: 'node_modules/**',
    }),
    replace({
      exclude: 'node_modules/**',
      ENV: JSON.stringify(process.env.NODE_ENV || 'development'),
    }),
  ],
};
```

在我们的配置中，将找打所有出现的ENV并且替换成 `process.env.NODE_ENV` - 在Node应用中最普遍的设置环境变量的方法 - 或者 "development"中的一个。使用JSON.stringify()确保值被双引号包裹，如果ENV没有的话。

为了确保不会和三方代码造成问题，同样设置exclude属性来忽略node_modules目录和其中的全部包。(幸亏@wesleycoder先在这上面踩坑了。)

### 检查结果

首先，重新打包然后在浏览器中检查。控制台日志会显示，就像之前一样。很棒 - 这意味着我们的默认值生效了。

为了展示新引入的能力，我们在 `production` 模式下运行命令：

```bash
NODE_ENV=production ./node_modules/.bin/rollup -c
```

> 注意: 在 Windows 上，使用 `SET NODE_ENV=production ./node_modules/.bin/rollup -c` 防止在设置环境变量时报错。

当刷新浏览器后，控制台没有任何日志打出了：



不改变任何代码的情况下，使用一个环境变量禁用了日志插件。

## 添加UglifyJS压缩减小生成代码体积

这个教程中最后一步是添加UglifyJS来减小和压缩bundle文件。可以通过移除注释，缩短变量名和其他压缩换行等方式大幅度减少bundle的大小 - 会让文件的可读性变差，但提高了网络间传输的效率。

### 安装插件

我们将使用 UglifyJS 压缩 bundle，通过 rollup-plugin-uglify 插件。

通过下面命令安装：

```bash
npm install --save-dev rollup-plugin-uglify
```

### 更新rollup.config.js

然后添加Uglify到Rollup配置。为了开发环境下可读性更好，设置代码丑化仅在生产环境下使用：

```js
// Rollup plugins
import babel from 'rollup-plugin-babel';
import eslint from 'rollup-plugin-eslint';
import resolve from 'rollup-plugin-node-resolve';
import commonjs from 'rollup-plugin-commonjs';
import replace from 'rollup-plugin-replace';
import uglify from 'rollup-plugin-uglify';

export default {
  entry: 'src/scripts/main.js',
  dest: 'build/js/main.min.js',
  format: 'iife',
  sourceMap: 'inline',
  plugins: [
    resolve({
      jsnext: true,
      main: true,
      browser: true,
    }),
    commonjs(),
    eslint({
      exclude: [
        'src/styles/**',
      ]
    }),
    babel({
      exclude: 'node_modules/**',
    }),
    replace({
      ENV: JSON.stringify(process.env.NODE_ENV || 'development'),
    }),
    (process.env.NODE_ENV === 'production' && uglify()),
  ],
};
```

我们使用了短路运算，很常用(虽然也有争议)的条件性设置值的方法。[注4]

在我们的例子中，只有在 NODE_ENV 是 "production" 时才会加载 uglify()。

### 检查压缩过的bundle

保存配置文件，让我们在生成环境下运行Rollup：

```bash
NODE_ENV=production ./node_modules/.bin/rollup -c
```

> 注意: 在Windows上，使用 `SET NODE_ENV=production ./node_modules/.bin/rollup -c` 防止在设置环境变量时报错。

输出内容并不美观，但是更小了。这有build/js/main.min.js的截屏，看起来像这样：



丑化过的代码确实能更高效地传输。

之前，我们的 bundle 大约42KB。使用 UglifyJS 后，减少到大约 29KB - 在没做其他优化的情况下节省了超过30%文件大小。

### 接下来的内容

在这个系列的下一节，我们将了解通过 Rollup 和 PostCSS 处理样式，并且添加 live reloading来实时看见我们的修改。

### 扩展阅读

The cost of small modules - 这篇文章让我开始对Rollup感兴趣，因为它展示了一些Rollup相比webpack和Browserify的优势。

Rollup’s getting started guide
Rollup’s CLI docs
A list of Rollup plugins

疑问？想法？指出Bug？

这篇文章的代码放在GitHub上。你可以fork 这个仓库进行修改或测试，开issue或者报告bug，或者新建 pull request 进行建议或者修改。

本文根据@jlengstorf的《Tutorial: How to Bundle JavaScript With Rollup》所译，整个译文带有我们自己的理解与思想，如果译得不好或有不对之处还请同行朋友指点。如需转载此译文，需注明英文出处:https://code.lengstorf.com/learn-rollup-js/。

著作权归作者所有。
商业转载请联系作者获得授权,非商业转载请注明出处。
原文: http://www.w3cplus.com/javascript/learn-rollup-js.html © w3cplus.com